In [1]:
#load libraries and the classification model

import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import shutil
from PIL import Image
from tensorflow.keras.models import load_model
model = load_model('avo_model.keras')
from ultralytics import YOLO


C:\Users\johnl\AppData\Local\Temp\ipykernel_11672\1336455791.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


First we run the yolo model to identify the kind of avo. Since we can only predict Hass, maybe I need something to account for this. 

Next we move these images into a separate checking folder. This is to make the rest of the code sustainable and work across all devices consistently. This will move all the Hass crops into a separate checking folder (or shepards at a later date, may need an if else or something) and then delete the runs folder to avoid having the future cops go into 'predict2' and then having to keep track of the number of predicts.

In [9]:
model_yolo = YOLO("avoID.pt")

model_yolo.predict(
        source='Pictures/1024A',
        save_crop=True,
        save=True,
        conf=0.5
    )
#!yolo predict model=avoID.pt source='Pictures/1020' conf = 0.5 save_crop=True save
shutil.move('runs/detect/predict/crops/Hass','checking_folder/') #need to add a line if there are no hass' detected
shutil.rmtree('runs/')


image 1/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_0.jpg: 640x512 1 Hass, 14.3ms
image 2/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_1.jpg: 640x512 1 Hass, 10.0ms
image 3/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_2.jpg: 640x512 1 Hass, 13.1ms
image 4/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_3.jpg: 640x512 1 Hass, 13.3ms
image 5/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_4.jpg: 640x512 1 Hass, 12.3ms
image 6/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_5.jpg: 640x512 1 Hass, 9.5ms
image 7/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1024A\1024A_6.jpg: 640x512 2 Hasss, 13.1ms
Speed: 2.0ms preprocess, 12.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 512)
Results saved to runs\detect\predict


The next steps: 
1) Define a function to resize the images inputed
2) open checking_folder and store the names of each file in a list
3) loop through that list and resize each image
4) run the image through the algo and save the score
5) get an average of the final score as the score for the avos

In [6]:
#This is to resize the image prior to running it through the model
def prepare_input(image_path, target_size=(256, 256)): #adjust target size to match model input.
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    return img_array

avo_pics = os.listdir('checking_folder/')

avo_score = [] #empty list to store the avo score

#Loop through all the pics in the folder to get the individual predictions for each avo
for pic in avo_pics:
    avo = prepare_input('checking_folder/' + pic)
    avo_score.append(model.predict(avo))

shutil.rmtree('checking_folder/') #delete the checking_folder to ensure we can use exactly the same code in the futur

avo_score = np.concatenate(avo_score, axis=0 )
avo_score.mean()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


3.16428

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_lite = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\johnl\AppData\Local\Temp\tmpmfz11kke\assets


INFO:tensorflow:Assets written to: C:\Users\johnl\AppData\Local\Temp\tmpmfz11kke\assets


Saved artifact at 'C:\Users\johnl\AppData\Local\Temp\tmpmfz11kke'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2254585538384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585540880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585538960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585539152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585541264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585542416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585543184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585544528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585542992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2254585543376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  225458

In [ ]:
##Lite model testing

##Ok this works well and is a little faster

def prepare_input(image_path, target_size=(256, 256)):
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

model_lite = tf.lite.Interpreter(model_path="avo_model_lite.tflite")
model_lite.allocate_tensors()
input_details = model_lite.get_input_details()
output_details = model_lite.get_output_details()

def predict_tflite(input_data):
    model_lite.set_tensor(input_details[0]['index'], input_data)
    model_lite.invoke()
    return model_lite.get_tensor(output_details[0]['index'])

avo_pics = os.listdir('checking_folder/')

avo_score = [] #empty list to store the avo score

#Loop through all the pics in the folder to get the individual predictions for each avo
for pic in avo_pics:
    avo = prepare_input('checking_folder/' + pic)
    avo_score.append(predict_tflite(avo))

shutil.rmtree('checking_folder/') #delete the checking_folder to ensure we can use exactly the same code in the futur

avo_score = np.concatenate(avo_score, axis=0 )
avo_score.mean()

3.164278